In [1]:
import pandas as pd
import polars as pl
import numpy as np
import lightgbm as lgb
import optuna

In [2]:
# base_path = '/home/clas_giulia_s/buckets/b1/'
base_path = '/Users/ignacio/MAESTRIA/DMEF/'

dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
# dataset_file = 'competencia_02.parquet'
dataset_file = 'competencia_01_fe_modelito.csv'

ganancia_acierto = 273000
costo_estimulo = 7000

In [3]:
semillas = [5623, 292, 7494, 8504, 1663, 785, 5377, 4838, 2141, 2235, 9836, 1258, 3273, 8349, 1639, 1597, 3195, 40, 5186, 9278, 6281, 7515, 2046, 5642, 505, 4611, 3008, 2063, 2280, 1148, 618, 4806, 1503, 3926, 6363, 400, 2662, 9432, 1632, 386, 2545, 228, 1561, 3523, 4508, 9190, 8181, 7302, 6250, 7762, 8141, 6854, 622, 5327, 6379, 3867, 5420, 3030, 7275, 2040, 6042, 4365, 231, 8330, 8527, 2420, 2558, 9618, 3937, 555, 122, 4907, 7838, 5246, 100, 3243, 1449, 1052, 1906, 7657, 753, 4320, 4576, 9621, 8868, 8155, 7410, 2320, 6355, 1994, 7775, 8358, 3508, 3064, 3904, 3602, 5308, 6947, 1544, 624]

In [4]:
# data = pl.scan_parquet(dataset_path + dataset_file).filter(pl.col("foto_mes") == 202108).collect()

# data = pl.scan_csv(dataset_path + dataset_file).filter(pl.col("foto_mes") == 202106).collect()

data = pd.read_csv(dataset_path + dataset_file)
data = data[data["foto_mes"] == 202106] 

In [5]:
# # convierto a pandas dataframe
# data = data.to_pandas()

In [6]:
# Asignamos pesos a las clases

data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [7]:
data['clase_binaria'] = 0
data['clase_binaria'] = np.where(data['clase_ternaria'] == 'BAJA+2', 1, 0)

In [8]:
X_test = data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1)

### Predicciones para Kaggle

Se generan la prediccion para cada modelo individual, se promedian las predicciones y luego se ordenan de mayor a menor para determinar el numero de envios

In [9]:
version = 'modelito' # UPADTE

modelos = [f'lgb_competencia1_{version}_s{semilla}_final.txt' for semilla in semillas]

In [10]:
len(modelos)

100

In [11]:
X_test_with_predictions = data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1).copy()

In [12]:
counter = 0

for modelo in modelos:
    
    # cargamos el modelo
    model = lgb.Booster(model_file=f'{modelos_path}{version}/{modelo}')
    
    # predecimos para agosto
    X_test = data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1).copy()
    y_pred_lgm = model.predict(X_test)
    X_test_with_predictions[f'pred_lgm_{semillas[counter]}'] = y_pred_lgm
    
    print(f'{counter}. Predicciones del modelo: {modelo} DONE')
    
    counter += 1

0. Predicciones del modelo: lgb_competencia1_modelito_s5623_final.txt DONE
1. Predicciones del modelo: lgb_competencia1_modelito_s292_final.txt DONE
2. Predicciones del modelo: lgb_competencia1_modelito_s7494_final.txt DONE
3. Predicciones del modelo: lgb_competencia1_modelito_s8504_final.txt DONE
4. Predicciones del modelo: lgb_competencia1_modelito_s1663_final.txt DONE
5. Predicciones del modelo: lgb_competencia1_modelito_s785_final.txt DONE
6. Predicciones del modelo: lgb_competencia1_modelito_s5377_final.txt DONE
7. Predicciones del modelo: lgb_competencia1_modelito_s4838_final.txt DONE
8. Predicciones del modelo: lgb_competencia1_modelito_s2141_final.txt DONE
9. Predicciones del modelo: lgb_competencia1_modelito_s2235_final.txt DONE
10. Predicciones del modelo: lgb_competencia1_modelito_s9836_final.txt DONE
11. Predicciones del modelo: lgb_competencia1_modelito_s1258_final.txt DONE
12. Predicciones del modelo: lgb_competencia1_modelito_s3273_final.txt DONE
13. Predicciones del mod

/var/folders/6c/1g3b2t8s1l5ch_c9nch8gppw0000gn/T/ipykernel_11696/3353481977.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test_with_predictions[f'pred_lgm_{semillas[counter]}'] = y_pred_lgm


98. Predicciones del modelo: lgb_competencia1_modelito_s1544_final.txt DONE
99. Predicciones del modelo: lgb_competencia1_modelito_s624_final.txt DONE


/var/folders/6c/1g3b2t8s1l5ch_c9nch8gppw0000gn/T/ipykernel_11696/3353481977.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test_with_predictions[f'pred_lgm_{semillas[counter]}'] = y_pred_lgm


In [13]:
X_test_with_predictions = X_test_with_predictions.copy()

In [14]:
individual_predictions = X_test_with_predictions.iloc[:, -100:].columns.tolist()
individual_predictions

['pred_lgm_5623',
 'pred_lgm_292',
 'pred_lgm_7494',
 'pred_lgm_8504',
 'pred_lgm_1663',
 'pred_lgm_785',
 'pred_lgm_5377',
 'pred_lgm_4838',
 'pred_lgm_2141',
 'pred_lgm_2235',
 'pred_lgm_9836',
 'pred_lgm_1258',
 'pred_lgm_3273',
 'pred_lgm_8349',
 'pred_lgm_1639',
 'pred_lgm_1597',
 'pred_lgm_3195',
 'pred_lgm_40',
 'pred_lgm_5186',
 'pred_lgm_9278',
 'pred_lgm_6281',
 'pred_lgm_7515',
 'pred_lgm_2046',
 'pred_lgm_5642',
 'pred_lgm_505',
 'pred_lgm_4611',
 'pred_lgm_3008',
 'pred_lgm_2063',
 'pred_lgm_2280',
 'pred_lgm_1148',
 'pred_lgm_618',
 'pred_lgm_4806',
 'pred_lgm_1503',
 'pred_lgm_3926',
 'pred_lgm_6363',
 'pred_lgm_400',
 'pred_lgm_2662',
 'pred_lgm_9432',
 'pred_lgm_1632',
 'pred_lgm_386',
 'pred_lgm_2545',
 'pred_lgm_228',
 'pred_lgm_1561',
 'pred_lgm_3523',
 'pred_lgm_4508',
 'pred_lgm_9190',
 'pred_lgm_8181',
 'pred_lgm_7302',
 'pred_lgm_6250',
 'pred_lgm_7762',
 'pred_lgm_8141',
 'pred_lgm_6854',
 'pred_lgm_622',
 'pred_lgm_5327',
 'pred_lgm_6379',
 'pred_lgm_3867',
 '

### Semillerio 100

In [15]:
X_test_with_predictions['predicted_avg'] = X_test_with_predictions.iloc[:, -100:].mean(axis=1)

In [16]:
# ordeno de mayor probabilidad de baja a menor
idx = np.argsort(X_test_with_predictions['predicted_avg'])[::-1]
X_test_with_predictions.reset_index(drop=True, inplace=True)
X_test_with_predictions = X_test_with_predictions.iloc[idx]

Se prueban varios puntos de corte

In [17]:
puntos_corte = np.linspace(9000, 13000, 50).astype(int)
puntos_corte

array([ 9000,  9081,  9163,  9244,  9326,  9408,  9489,  9571,  9653,
        9734,  9816,  9897,  9979, 10061, 10142, 10224, 10306, 10387,
       10469, 10551, 10632, 10714, 10795, 10877, 10959, 11040, 11122,
       11204, 11285, 11367, 11448, 11530, 11612, 11693, 11775, 11857,
       11938, 12020, 12102, 12183, 12265, 12346, 12428, 12510, 12591,
       12673, 12755, 12836, 12918, 13000])

In [ ]:
counter = 0

for corte in puntos_corte:
    
    envios = np.zeros(len(X_test_with_predictions), dtype=int)
    envios[:corte] = 1
    X_test_with_predictions['Predicted'] = envios
    
    output = X_test_with_predictions[['numero_de_cliente', 'Predicted']]
        
    counter += 1
    
    file_name = f'results_{version}_{counter}.csv'
    output_path = base_path + f'exp/competencia_1/{version}/' + file_name
    
    output.to_csv(output_path, index=False)
            
    print(f'{counter}. corte: {corte}, {output_path}')

1. corte: 9000, /Users/ignacio/MAESTRIA/DMEF/exp/competencia_1/modelito/results_modelito_1.csv
2. corte: 9081, /Users/ignacio/MAESTRIA/DMEF/exp/competencia_1/modelito/results_modelito_2.csv
3. corte: 9163, /Users/ignacio/MAESTRIA/DMEF/exp/competencia_1/modelito/results_modelito_3.csv
4. corte: 9244, /Users/ignacio/MAESTRIA/DMEF/exp/competencia_1/modelito/results_modelito_4.csv
5. corte: 9326, /Users/ignacio/MAESTRIA/DMEF/exp/competencia_1/modelito/results_modelito_5.csv
6. corte: 9408, /Users/ignacio/MAESTRIA/DMEF/exp/competencia_1/modelito/results_modelito_6.csv
7. corte: 9489, /Users/ignacio/MAESTRIA/DMEF/exp/competencia_1/modelito/results_modelito_7.csv
8. corte: 9571, /Users/ignacio/MAESTRIA/DMEF/exp/competencia_1/modelito/results_modelito_8.csv
9. corte: 9653, /Users/ignacio/MAESTRIA/DMEF/exp/competencia_1/modelito/results_modelito_9.csv
10. corte: 9734, /Users/ignacio/MAESTRIA/DMEF/exp/competencia_1/modelito/results_modelito_10.csv
11. corte: 9816, /Users/ignacio/MAESTRIA/DMEF/ex

In [ ]:

# predicciones_list = []

# # Recorre los archivos generados y carga las predicciones
# for counter in range(1, len(puntos_corte) + 1):
#     file_name = f'results_{version}_{counter}.csv'
#     output_path = base_path + f'exp/competencia_1/{version}/' + file_name
    
#     # Cargar las predicciones desde el archivo CSV
#     df = pd.read_csv(output_path)
#     predicciones_list.append(df['Predicted'].values)

# # Convertir la lista de predicciones a un array de numpy
# predicciones_array = np.array(predicciones_list)

# # Calcular el promedio de las predicciones
# promedio_predicciones = np.mean(predicciones_array, axis=0)

# # Crear un DataFrame con el promedio de las predicciones
# X_test_with_predictions['Predicted'] = promedio_predicciones

# # Guardar el promedio de las predicciones en un archivo CSV
# promedio_output = X_test_with_predictions[['numero_de_cliente', 'Predicted']]
# promedio_output_path = base_path + f'exp/competencia_1/{version}/promedio_results.csv'
# promedio_output.to_csv(promedio_output_path, index=False)

# print(f'Promedio de predicciones guardado en: {promedio_output_path}')

Promedio de predicciones guardado en: /Users/ignacio/MAESTRIA/DMEF/exp/competencia_1/modelito/promedio_results.csv


In [33]:

predicciones_list = []

# Recorre los archivos generados y carga las predicciones
for counter in range(1, len(puntos_corte) + 1):
    file_name = f'results_{version}_{counter}.csv'
    output_path = base_path + f'exp/competencia_1/{version}/' + file_name
    
    # Cargar las predicciones desde el archivo CSV
    df = pd.read_csv(output_path)
    predicciones_list.append(df['Predicted'].values)

# Convertir la lista de predicciones a un array de numpy
predicciones_array = np.array(predicciones_list)

# Calcular el promedio de las predicciones
promedio_predicciones = np.mean(predicciones_array, axis=0)

# Redondear cualquier número mayor que 0.25 a 1 y convertir a enteros
promedio_predicciones = np.where(promedio_predicciones > 0.25, 1, 0).astype(int)

# Crear un DataFrame con el promedio de las predicciones
X_test_with_predictions['Predicted'] = promedio_predicciones

# Guardar el promedio de las predicciones en un archivo CSV
promedio_output = X_test_with_predictions[['numero_de_cliente', 'Predicted']]
promedio_output_path = base_path + f'exp/competencia_1/{version}/promedio_up_results.csv'
promedio_output.to_csv(promedio_output_path, index=False)

print(f'Promedio de predicciones guardado en: {promedio_output_path}')



Promedio de predicciones guardado en: /Users/ignacio/MAESTRIA/DMEF/exp/competencia_1/modelito/promedio_up_results.csv


In [ ]:
# # Subir el archivo con el promedio de las predicciones a Kaggle
# !kaggle competitions submit -c <competition-name> -f {promedio_output_path} -m "Submission with averaged predictions"

# print(f'Subida a Kaggle: {promedio_output_path}')